# Crypto_Data Notebook
This is a rough draft notebook used to design and test crypto_data.py module functions

In [14]:
# Importing necessary libraries
import pandas as pd
from pycoingecko import CoinGeckoAPI
import datetime
import time 
cg = CoinGeckoAPI()

In [3]:
# Create start and end time for CoinGeckoAPI
user_start_time = '2013-01-01'
user_end_time = '2021-05-14'

cg_start = datetime.datetime.strptime(user_start_time, "%Y-%m-%d")
cg_start = datetime.datetime.timestamp(cg_start)
cg_start = str(cg_start)

# if statement for user inputs now for present time, else they will input string in YYYY-mm-dd
if user_end_time == 'now':
    cg_end = datetime.datetime.now() #can allow users to enter in date time
    cg_end = datetime.datetime.timestamp(cg_end)
    cg_end = str(cg_end)
    
else:
    cg_end = datetime.datetime.strptime(user_end_time, "%Y-%m-%d")
    cg_end = datetime.datetime.timestamp(cg_end)
    cg_end = str(cg_end)

#testing cg_start and cg_end variables for UNIX format
print(f'''UNIX start: {cg_start}      type{type(cg_start)}
UNIX end: {cg_end}      type {type(cg_end)}''') 

UNIX start: 1357027200.0      type<class 'str'>
UNIX end: 1620975600.0      type <class 'str'>


In [4]:
# List of dictionaries for each crypto...keys are 'id', 'symbol' etc...max per page is 250 so to get 500 coins and save results to variables cg_1, cg_2
cg_1 = cg.get_coins_markets(order='market_cap_desc', vs_currency='usd', per_page=250, page=1)
cg_2 = cg.get_coins_markets(order='market_cap_desc', vs_currency='usd', per_page=250, page=2)

#loop through data from CoinGecko stored in `cg_1` and `cg_2` using `cg.get_coin_markets` and save to list `coin_list` becasue limit of 
coin_list = []
for coin in cg_1:
    coin_list.append([coin['id'], coin['symbol'], coin['name'], coin['image'], coin['market_cap'], coin['market_cap_rank']])
    
for coin in cg_2:
    coin_list.append([coin['id'], coin['symbol'], coin['name'], coin['image'], coin['market_cap'], coin['market_cap_rank']])
    
coin_list_df = pd.DataFrame(coin_list, columns=['CG id','CG symbol', 'name', 'image', 'market_cap', 'market cap rank'])
coin_list_df
coin_list_df.to_csv('coin_list.csv')

# This is how to look up time when file is saved: https://stackoverflow.com/questions/237079/how-to-get-file-creation-modification-date-times-in-python
# can use this for decide if making api call is necessary

In [21]:
loop to create data frame from .csv file
coin_list_df = pd.read_csv('coin_list.csv')
# coin_list_df['CG id'][0:49] 
prices_dictionary = {}
df_list=[]#create an empty data frame list https://stackoverflow.com/questions/28910089/filling-empty-python-dataframe-using-loops


for CG_id in coin_list_df['CG id']:
    time.sleep(0.65) #adds a 0.65 second delay every call staying with in CoinGecko 100 calls/min (60 secs/100 calls) limit https://realpython.com/python-sleep/
    coin_API = cg.get_coin_market_chart_range_by_id( CG_id, 'usd', cg_start, cg_end) #API call to CoinGecko and grabing historical price data
    prices_dictionary[CG_id] = coin_API['prices'] # updating dictionary with CG_id as key and list o lists of unix time stamp and price data
    df_temp = pd.DataFrame(prices_dictionary[CG_id], columns=['time', CG_id+' prices']) #creating a temporary df that will later be appeneded into 
    df_temp['time'] = pd.to_datetime(df_temp['time'], origin='unix', unit='ms') #links (was having issue converting UNIX this helped https://github.com/pandas-dev/pandas/issues/10987)
    df_temp.set_index('time', inplace=True)
    df_list.append(df_temp)#append dataframe list in the loop
    
coin_historical_df = pd.concat(df_list, axis=1, join='outer') 
coin_historical_df.to_csv('crypto_price_data.csv')
# df['time'] = pd.to_datetime(df['time'], origin='unix', unit='ms') #links (was having issue converting UNIX this helped https://github.com/pandas-dev/pandas/issues/10987)

coin_historical_df
#LEFT OFF HERE...NEED TO FIGURE OUT HOW TO MAKE DICTIONARY WITH VALUES AS LIST OF LISTS INTO A DATAFRAME


,bitcoin prices,ethereum prices,binancecoin prices,cardano prices,ripple prices,dogecoin prices,tether prices,polkadot prices,bitcoin-cash prices,litecoin prices,...,metronome prices,zb-token prices,gochain prices,dego-finance prices,wault-finance-old prices,mcdex prices,everid prices,btu-protocol prices,revv prices,sbtc prices
time,,,,,,,,,,,,,,,,,,,,,
2013-04-28,135.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.299830,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-29,141.960000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.359400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-30,135.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.182950,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-05-01,117.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.649140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-05-02,103.430000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.388790,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-10,58213.926294,3932.754068,661.891709,1.774032,1.533176,0.573136,1.001046,40.305088,1425.247161,384.671618,...,5.303168,0.573007,0.054212,14.080691,96.673092,27.014536,0.657449,0.739948,0.250148,60114.990195
2021-05-11,55901.667595,3979.608652,634.589453,1.651143,1.399744,0.451129,1.007187,36.697577,1343.418820,361.765245,...,5.551172,0.548503,0.056867,12.553337,113.917339,27.092437,0.719392,0.757164,0.254743,56907.980201
2021-05-12,56928.972357,4182.790286,672.637825,1.767170,1.476666,0.496731,1.000726,38.629214,1547.082465,377.688779,...,5.399746,0.567298,0.058589,13.356040,96.302238,27.920935,0.654896,0.740076,0.257986,58028.581686


In [ ]:
help(ts)